In [1]:
from borealis.modeling import BorealisForConditionalGeneration
from borealis.dataset import BorealisBaseDataset
import torch
from transformers import AutoTokenizer, WhisperFeatureExtractor
from tqdm.auto import tqdm
from datasets import load_dataset, Audio

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")

start_audio_token = "<|start_of_audio|>"
end_audio_token = "<|end_of_audio|>"

tokenizer.add_special_tokens(
    {"additional_special_tokens": [start_audio_token, end_audio_token]}
)

2

In [3]:
ds = load_dataset("Vikhrmodels/ToneSpeak", cache_dir="../cache_home/")
ds = ds.cast_column("audio", Audio(decode=True, sampling_rate=16_000))

whisper_encoder = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v3")

In [4]:
model = BorealisForConditionalGeneration(tokenizer=tokenizer)

In [5]:
model.load_state_dict(
    torch.load("/workspace/Borealis/asr_qwen_ckpts/checkpoint-17247/pytorch_model.bin")
)

<All keys matched successfully>

In [6]:
model = model.to("cuda:0")

In [8]:
eval_dataset = BorealisBaseDataset(
    audio_processor=whisper_encoder,
    text_tokenizer=tokenizer,
    audios=ds["validation"]["audio"][:4],
    texts=ds["validation"]["text"][:4],
    max_text_len=320,
)

In [9]:
generated_transcripts = []
ground_truth_texts = []

with torch.inference_mode():
    for batch in tqdm(eval_dataset, desc="Inference on eval set"):
        mel = batch["mel"].to(model.encoder.device)  # (B, 128, 3000)
        att_mask = batch["audio_att_mask"].to(model.encoder.device)  # (B, 3000)

        transcripts = model.generate(
            mel=mel,
            att_mask=att_mask,
            max_new_tokens=320,
            do_sample=True,
            top_p=0.9,
            top_k=50,
        )
        print(transcripts)

        gt_texts = tokenizer.decode(batch["labels"], skip_special_tokens=True)

        print(gt_texts)

        generated_transcripts.extend(transcripts)
        ground_truth_texts.extend(gt_texts)


for i in range(min(5, len(generated_transcripts))):
    print(f"Сгенерировано: {generated_transcripts[i]}")
    print(f"Правильный текст: {ground_truth_texts[i]}")
    print("-" * 80)

Inference on eval set:   0%|          | 0/4 [00:00<?, ?it/s]

Сегодня утром, когда ветер ласково трепал листьями на деревьях, я решила прогуляться по парку и насладиться свежим воздухом, который наполнял душу, спокойствием и радостью.
Сегодня утром, когда ветер ласково трепал листья на деревьях, я решила прогуляться по парку и насладиться свежим воздухом, который наполнял душу спокойствием и радостью.
Вечерние огни города плавно загораются, отражаясь в спокойной воде реки, создавая чарующую атмосферу уйтра и тихого умиротворения для всех, кто прогуливается по набережной.
Вечерние огни города плавно загораются, отражаясь в спокойной воде реки, создавая чарующую атмосферу уюта и тихого умиротворения для всех, кто прогуливается по набережной.
Сегодня утром солнце ярко сияло над горизонтом, освещая зеленые поля и пробуждая в туше легкое чувство радости и умиротворения.
Сегодня утром солнце ярко сияло над горизонтом, освещая зелёные поля и пробуждая в душе лёгкое чувство радости и умиротворения.
Вечерняя прогулка по парку всегда приносит особое чувств

In [10]:
import torch
import torchaudio


def test_single_wav(
    wav_path: str,
    model: BorealisForConditionalGeneration,
    audio_processor: WhisperFeatureExtractor,
    max_seconds_len: float = 30.0,
    sampling_rate: int = 16000,
    device: str = "cuda" if torch.cuda.is_available() else "cpu",
    generation_params: dict = None,
):
    """
    Загружает один WAV файл, обрабатывает его и генерирует транскрипт с помощью модели.

    :param wav_path: Путь к WAV файлу.
    :param model: Инстанс WhisperQWenASRModel.
    :param audio_processor: WhisperFeatureExtractor для обработки аудио.
    :param max_seconds_len: Максимальная длина аудио в секундах (для паддинга).
    :param sampling_rate: Частота дискретизации (по умолчанию 16000).
    :param device: Устройство ('cuda' или 'cpu').
    :param generation_params: Словарь с параметрами для model.generate (например, {'temperature': 0.0, 'max_new_tokens': 320}).
    """

    model.eval()
    model.to(device)

    waveform, sr = torchaudio.load(wav_path)
    if sr != sampling_rate:
        resampler = torchaudio.transforms.Resample(sr, sampling_rate)
        waveform = resampler(waveform)

    waveform = waveform.mean(dim=0).numpy()  # (T,)

    proc = audio_processor(
        waveform,
        sampling_rate=sampling_rate,
        padding="max_length",
        max_length=int(max_seconds_len * sampling_rate),
        return_attention_mask=True,
        return_tensors="pt",
    )
    mel = proc.input_features.squeeze(0).to(device)  # (80, 3000)
    att_mask = proc.attention_mask.squeeze(0).to(device)  # (3000)

    if generation_params is None:
        generation_params = {
            "max_new_tokens": 320,
            "do_sample": True,
            "top_p": 0.9,
            "top_k": 50,
        }

    with torch.inference_mode():
        transcript = model.generate(mel=mel, att_mask=att_mask, **generation_params)

    print(f"Generated transcript for {wav_path}:")
    print(transcript)

In [11]:
test_single_wav(
    wav_path="/workspace/res.wav",
    model=model,
    audio_processor=whisper_encoder,
    generation_params={
        "top_p": 0.9,
        "do_sample": True,
        "max_new_tokens": 320,
        "temperature": 0.95,
    },
)

Generated transcript for /workspace/res.wav:
Смехал грека через реку, видит грека в реку, рак за руку греку, цап!


In [12]:
test_ds = load_dataset("Vikhrmodels/ToneRuLS")

Using the latest cached version of the dataset since Vikhrmodels/ToneRuLS couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/Vikhrmodels___tone_ru_ls/default/0.0.0/4f7ee71a5b072597dc935538a82c958c80f9699c (last modified on Wed Jul 23 15:46:50 2025).


Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

In [13]:
test_ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'text_with_preprocessing'],
        num_rows: 53218
    })
    validation: Dataset({
        features: ['audio', 'text', 'text_with_preprocessing'],
        num_rows: 4006
    })
})

In [26]:
eval_dataset = BorealisBaseDataset(
    audio_processor=whisper_encoder,
    text_tokenizer=tokenizer,
    audios=test_ds["validation"]["audio"][:79],
    texts=test_ds["validation"]["text"][:79],
    max_text_len=320,
)

In [27]:
from jiwer import wer, cer

In [28]:
next(iter(eval_dataset))

{'mel': tensor([[ 0.0308, -0.4614, -0.4598,  ..., -0.8413, -0.8413, -0.8413],
         [ 0.1283, -0.3639, -0.3623,  ..., -0.8413, -0.8413, -0.8413],
         [ 0.2090, -0.0085, -0.0587,  ..., -0.8413, -0.8413, -0.8413],
         ...,
         [-0.8413, -0.8413, -0.8413,  ..., -0.8413, -0.8413, -0.8413],
         [-0.8413, -0.8413, -0.8413,  ..., -0.8413, -0.8413, -0.8413],
         [-0.8413, -0.8413, -0.8413,  ..., -0.8413, -0.8413, -0.8413]]),
 'audio_att_mask': tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.int32),
 'labels': tensor([ 20195,   8178,     11, 126081, 127287,  18943, 139732,     11,   8215,
         137477, 134378,  45310,  10813, 128399,  46195,   5805,  12121,  18492,
             11,  18658,  85191,   4824, 125469, 126491,  11310, 126118,     11,
         126491,  57217,  31885,  30343,     11, 126491,  54517,  19077,  16748,
             11, 126491,  11310,   7336,  22496,   1802,     13, 151645, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643,

In [29]:
generated_transcripts = []
ground_truth_texts = []

with torch.inference_mode():
    for batch in tqdm(eval_dataset, desc="Inference on eval set"):
        mel = batch["mel"].to(model.encoder.device)  # (B, 128, 3000)
        att_mask = batch["audio_att_mask"].to(model.encoder.device)  # (B, 3000)

        transcripts = model.generate(
            mel=mel,
            att_mask=att_mask,
            max_new_tokens=320,
            do_sample=True,
            top_p=0.9,
            top_k=50,
        )

        gt_texts = tokenizer.decode(batch["labels"], skip_special_tokens=True)

        generated_transcripts.append(transcripts)
        ground_truth_texts.append(gt_texts)


Inference on eval set:   0%|          | 0/79 [00:00<?, ?it/s]

In [35]:
import string

def clean_text_list(text_list):
    # Создаем множество символов пунктуации
    punct = set(string.punctuation)
    
    # Обрабатываем каждый текст в списке
    cleaned_list = [
        ''.join(char for char in text.lower() if char not in punct)
        for text in text_list
    ]
    
    return cleaned_list

In [36]:
wer_score = wer(clean_text_list(ground_truth_texts), clean_text_list(generated_transcripts))
cer_score = cer(clean_text_list(ground_truth_texts), clean_text_list(generated_transcripts))

In [37]:
wer_score

0.21387832699619772

In [38]:
cer_score

0.10763052208835341

In [20]:
!ffmpeg -version

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ffmpeg version n7.0.2-189-gf98f142da5 Copyright (c) 2000-2024 the FFmpeg developers
built with gcc 11 (Ubuntu 11.4.0-1ubuntu1~22.04)
configuration: --enable-gpl --enable-libx264 --enable-libx265 --enable-libvpx --enable-libfdk-aac --enable-libmp3lame --enable-libopus --enable-libdav1d --enable-libass --enable-libfreetype --enable-sdl2 --enable-nonfree
libavutil      59.  8.100 / 59.  8.100
libavcodec     61.  3.100 / 61.  3.100
libavformat    61.  1.100 / 61.  1.100
libavdevice    61.  1.100 / 61.  1.100
libavfilter    10.  1.100 / 10.  1.100
libswscale      8.  1.100 /  8.  1.100
libswresample   5.  1.100 /  5.  1.100
libpostproc    58.  1.100 / 58.  1.100


In [18]:
import random
def test_random_sample(dataset):
    if len(dataset) == 0:
        print("Датасет пустой!")
        return
    
    index = random.randint(0, len(dataset) - 1)
    print(f"Тестируем сэмпл с индексом: {index}")
    
    try:
        sample = dataset[index]
        print("Сэмпл успешно загружен!")
        print("Ключи в сэмпле:", list(sample.keys()))
        
        # Печать форм (для отладки)
        print("Форма mel:", sample["mel"].shape)
        print("Форма audio_att_mask:", sample["audio_att_mask"].shape)
        print("Форма labels:", sample["labels"].shape)
        print("Форма text_att_mask:", sample["text_att_mask"].shape)
        
        # Опционально: декодировать текст для проверки
        decoded_text = dataset.tokenizer.decode(sample["labels"], skip_special_tokens=True)
        print("Декодированный текст:", decoded_text)
        
    except Exception as e:
        print(f"Ошибка при загрузке сэмпла {index}: {e}")
        # Если ошибка в конкретном файле, можно добавить больше отладки
        if "path" in dataset.audios[index]:
            print("Путь к аудио:", dataset.audios[index]["path"])

# Запуск теста
test_random_sample(eval_dataset)

Тестируем сэмпл с индексом: 3
Ошибка при загрузке сэмпла 3: The frame has 0 channels, expected 1. If you are hitting this, it may be because you are using a buggy FFmpeg version. FFmpeg4 is known to fail here in some valid scenarios. Try to upgrade FFmpeg?


TypeError: 'torchcodec.decoders.AudioDecoder' object is not subscriptable